IMPORTS

---





In [ ]:
import math
import re

import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

import tensorflow as tf

from tensorflow.keras import layers
import tensorflow_datasets as tfds

from sklearn.model_selection import train_test_split

DATASET

---



In [ ]:
columns = ["sentiment", "id", "date", "query", "user", "tweet"]

dataset = pd.read_csv('data.csv', 
                      header=None,
                      names=columns,
                      engine="python",
                      encoding="latin1",
                      error_bad_lines=False)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 1278130: unexpected end of data


In [ ]:
dataset.drop(["id", "date", "query", "user"],
          axis=1,
          inplace=True)

In [ ]:
dataset.head()

,sentiment,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


In [ ]:
dataset['sentiment'].unique()

array([0, 4])

In [ ]:
data_labels = dataset["sentiment"].values
data_labels[data_labels == 4] = 1

In [ ]:
dataset['sentiment'].unique()

array([0, 1])

DATA PREPROCESSING

---



TRAIN/TEST SPLIT

---



In [ ]:
X = dataset["tweet"]
y = dataset["sentiment"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

### SIMPLE NAIVE BAYES

MODEL CONFIGURATION

---



MODEL TRAINING

---



MODEL EVALUATION

---



NEW PREDICTIONS

---



### CONVOLUTIONAL NEURAL NETWORK

MODEL CONFIGURATION

---



MODEL TRAINING

---



MODEL EVALUATION

---



NEW PREDICTIONS

---



### RECURRENT NEURAL NETWORK

MODEL CONFIGURATION

---



MODEL TRAINING

---



MODEL EVALUATION

---



NEW PREDICTIONS

---

